# GENERACION POR TECNOLOGIA

In [1]:
# https://altair-viz.github.io/gallery/interactive_legend.html

import altair as alt
from vega_datasets import data

source = data.unemployment_across_industries.url

selection = alt.selection_point(fields=['series'], bind='legend')

alt.Chart(source).mark_area().encode(
    alt.X('yearmonth(date):T').axis(domain=False, format='%Y', tickSize=0),
    alt.Y('sum(count):Q').stack('center').axis(None),
    alt.Color('series:N').scale(scheme='category20b'),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2))
).add_params(
    selection
)

alt.Chart(...)

In [2]:
# https://altair-viz.github.io/gallery/multiple_interactions.html
# PARA EL SISTEMA DE SELECCION DE DIA A GRAFICAR

import altair as alt
from vega_datasets import data

movies = alt.UrlData(
    data.movies.url,
    format=alt.DataFormat(parse={"Release_Date":"date"})
)
ratings = ['G', 'NC-17', 'PG', 'PG-13', 'R']
genres = [
    'Action', 'Adventure', 'Black Comedy', 'Comedy',
    'Concert/Performance', 'Documentary', 'Drama', 'Horror', 'Musical',
    'Romantic Comedy', 'Thriller/Suspense', 'Western'
]

base = alt.Chart(movies, width=200, height=200).mark_point(filled=True).transform_calculate(
    Rounded_IMDB_Rating = "floor(datum.IMDB_Rating)",
    Big_Budget_Film =  "datum.Production_Budget > 100000000 ? 'Yes' : 'No'",
    Release_Year = "year(datum.Release_Date)",
).transform_filter(
    alt.datum.IMDB_Rating > 0
).transform_filter(
    alt.FieldOneOfPredicate(field='MPAA_Rating', oneOf=ratings)
).encode(
    x=alt.X('Worldwide_Gross:Q').scale(domain=(100000,10**9), clamp=True),
    y='IMDB_Rating:Q',
    tooltip="Title:N"
)

# A slider filter
year_slider = alt.binding_range(min=1969, max=2018, step=1, name="Release Year")
slider_selection = alt.selection_point(bind=year_slider, fields=['Release_Year'])

filter_year = base.add_params(
    slider_selection
).transform_filter(
    slider_selection
).properties(title="Slider Filtering")

# A dropdown filter
genre_dropdown = alt.binding_select(options=genres, name="Genre")
genre_select = alt.selection_point(fields=['Major_Genre'], bind=genre_dropdown)

filter_genres = base.add_params(
    genre_select
).transform_filter(
    genre_select
).properties(title="Dropdown Filtering")

# Color changing marks
rating_radio = alt.binding_radio(options=ratings, name="Rating")
rating_select = alt.selection_point(fields=['MPAA_Rating'], bind=rating_radio)

rating_color_condition = alt.condition(
    rating_select,
    alt.Color('MPAA_Rating:N').legend(None),
    alt.value('lightgray')
)

highlight_ratings = base.add_params(
    rating_select
).encode(
    color=rating_color_condition
).properties(title="Radio Button Highlighting")

# Boolean selection for format changes
input_checkbox = alt.binding_checkbox(name="Big Budget Films ")
checkbox_selection = alt.param(bind=input_checkbox)

size_checkbox_condition = alt.condition(
    checkbox_selection,
    alt.Size('Big_Budget_Film:N').scale(range=[25, 150]),
    alt.SizeValue(25)
)

budget_sizing = base.add_params(
    checkbox_selection
).encode(
    size=size_checkbox_condition
).properties(title="Checkbox Formatting")

(filter_year | budget_sizing) & (highlight_ratings | filter_genres)

alt.VConcatChart(...)

In [60]:
import pandas as pd
import numpy as np

potencia_solar = np.random.randint(10, 100, size=48)  # Potencias ficticias entre 10 y 100
potencia_eolica = np.random.randint(10, 100, size=48)  # Potencias ficticias entre 10 y 100
horas = pd.date_range("2023-01-01 00:00", periods=48, freq='H').time
dia_solar = ['2023-01-01'] * 24 + ['2023-01-02'] * 24

data = {
    'hora': horas,
    'dia': dia_solar,
    'potencia_solar': potencia_solar,
    'potencia_eolica': potencia_eolica,
}


# Convertir a DataFrame
df = pd.DataFrame(data)
df['hora'] = pd.to_datetime(df['dia'] + ' ' + df['hora'].astype(str))

In [61]:
# Desdoblar las columnas
df_melted = df.melt(id_vars=['hora', 'dia'], 
                    value_vars=['potencia_solar', 'potencia_eolica'], 
                    var_name='tecnologia', 
                    value_name='potencia')

# Reemplazar los valores de la columna 'tecnologia'
df_melted['tecnologia'] = df_melted['tecnologia'].replace({
    'potencia_solar': 'Solar',
    'potencia_eolica': 'Eólica'
})

# Mostrar el DataFrame resultante
print(df_melted)
df = df_melted 

                  hora         dia tecnologia  potencia
0  2023-01-01 00:00:00  2023-01-01      Solar        99
1  2023-01-01 01:00:00  2023-01-01      Solar        90
2  2023-01-01 02:00:00  2023-01-01      Solar        43
3  2023-01-01 03:00:00  2023-01-01      Solar        38
4  2023-01-01 04:00:00  2023-01-01      Solar        55
..                 ...         ...        ...       ...
91 2023-01-02 19:00:00  2023-01-02     Eólica        98
92 2023-01-02 20:00:00  2023-01-02     Eólica        29
93 2023-01-02 21:00:00  2023-01-02     Eólica        51
94 2023-01-02 22:00:00  2023-01-02     Eólica        57
95 2023-01-02 23:00:00  2023-01-02     Eólica        44

[96 rows x 4 columns]


In [63]:
# Extraer año, mes y día para la selección
df['anio'] = df['hora'].dt.year
df['mes'] = df['hora'].dt.month
df['dia'] = df['hora'].dt.day

In [69]:
print(df)

                  hora  dia tecnologia  potencia  anio  mes
0  2023-01-01 00:00:00    1      Solar        99  2023    1
1  2023-01-01 01:00:00    1      Solar        90  2023    1
2  2023-01-01 02:00:00    1      Solar        43  2023    1
3  2023-01-01 03:00:00    1      Solar        38  2023    1
4  2023-01-01 04:00:00    1      Solar        55  2023    1
..                 ...  ...        ...       ...   ...  ...
91 2023-01-02 19:00:00    2     Eólica        98  2023    1
92 2023-01-02 20:00:00    2     Eólica        29  2023    1
93 2023-01-02 21:00:00    2     Eólica        51  2023    1
94 2023-01-02 22:00:00    2     Eólica        57  2023    1
95 2023-01-02 23:00:00    2     Eólica        44  2023    1

[96 rows x 6 columns]


In [70]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   hora        96 non-null     datetime64[ns]
 1   dia         96 non-null     int32         
 2   tecnologia  96 non-null     object        
 3   potencia    96 non-null     int64         
 4   anio        96 non-null     int32         
 5   mes         96 non-null     int32         
dtypes: datetime64[ns](1), int32(3), int64(1), object(1)
memory usage: 3.5+ KB


In [87]:
# Crear selectores
selector_anio = alt.binding_select(options=list(df['anio'].unique()), name='Año:') #
selector_mes = alt.binding_select(options=list(range(1, 13)), name='Mes:')
selector_dia = alt.binding_select(options=list(range(1, 32)), name='Día:')

# Crear campos de selección
anio_select = alt.selection_point(fields=['anio'], bind=selector_anio, value=df['anio'].iloc[0]) #{'anio': df['anio'].iloc[0]})
mes_select = alt.selection_point(fields=['mes'], bind=selector_mes, value=df['mes'].iloc[0])
dia_select = alt.selection_point(fields=['dia'], bind=selector_dia, value=df['dia'].iloc[0])


# Crear selectores
selector_anio = alt.binding_select(options=list(df['anio'].unique()), name='Año:') #
selector_mes = alt.binding_select(options=list(range(1, 13)), name='Mes:')
selector_dia = alt.binding_select(options=list(range(1, 32)), name='Día:')

# Crear campos de selección
anio_select = alt.selection_interval(fields=['anio'], bind=selector_anio, value=df['anio'].iloc[0]) #{'anio': df['anio'].iloc[0]})
mes_select = alt.selection_interval(fields=['mes'], bind=selector_mes, value=df['mes'].iloc[0])
dia_select = alt.selection_interval(fields=['dia'], bind=selector_dia, value=df['dia'].iloc[0])

In [88]:
alt.Chart(df).mark_area(interpolate='step').encode(
    x=alt.X('hora:T', title='Hora del día'),  # Eje X: Hora del día
    y=alt.Y('sum(potencia):Q', title='Potencia eléctrica (MW)'),  # Suma de potencia
    color=alt.Color('tecnologia:N', title='Tecnología')  # Diferenciar por tecnología
).properties(
    title='Generación eléctrica por tecnología',
    width=800,
    height=400
).add_selection(anio_select, mes_select, dia_select).transform_filter(
    (alt.datum.anio == anio_select.anio) & 
    (alt.datum.mes == mes_select.mes) & 
    (alt.datum.dia == dia_select.dia)
)

# https://github.com/vega/altair/issues/2008

/tmp/ipykernel_3289145/1984300968.py:1: AltairDeprecationWarning: Deprecated in `altair=5.0.0`. Use add_params instead.
  alt.Chart(df).mark_area(interpolate='step').encode(


alt.Chart(...)

In [76]:
print(df[(df['anio'] == anio_select.anio) & 
          (df['mes'] == mes_select.mes) & 
          (df['dia'] == dia_select.dia)])

                  hora  dia tecnologia  potencia  anio  mes
0  2023-01-01 00:00:00    1      Solar        99  2023    1
1  2023-01-01 01:00:00    1      Solar        90  2023    1
2  2023-01-01 02:00:00    1      Solar        43  2023    1
3  2023-01-01 03:00:00    1      Solar        38  2023    1
4  2023-01-01 04:00:00    1      Solar        55  2023    1
..                 ...  ...        ...       ...   ...  ...
91 2023-01-02 19:00:00    2     Eólica        98  2023    1
92 2023-01-02 20:00:00    2     Eólica        29  2023    1
93 2023-01-02 21:00:00    2     Eólica        51  2023    1
94 2023-01-02 22:00:00    2     Eólica        57  2023    1
95 2023-01-02 23:00:00    2     Eólica        44  2023    1

[96 rows x 6 columns]


In [59]:
# Generar un gráfico de área acumulada
alt.Chart(df).mark_area(interpolate='step').encode(
    x=alt.X('hora:T', title='Hora del día'),  # Eje X: Hora del día
    y=alt.Y('sum(potencia):Q', title='Potencia eléctrica (MW)'),  # Suma de potencia
    color=alt.Color('tecnologia:N', title='Tecnología')  # Diferenciar por tecnología
).properties(
    title='Generación eléctrica por tecnología',
    width=800,
    height=400
)

alt.Chart(...)

In [79]:
import pandas as pd
import altair as alt

source = pd.DataFrame({"category": [1, 2, 3, 4, 5, 6], "value": [4, 6, 10, 3, 7, 8]})

alt.Chart(source).mark_arc().encode(
    theta="value",
    color="category"
)

alt.Chart(...)

In [77]:
import pandas as pd
import altair as alt

source = pd.DataFrame({"values": [12, 23, 47, 6, 52, 19]})

base = alt.Chart(source).encode(
    alt.Theta("values:Q").stack(True),
    alt.Radius("values").scale(type="sqrt", zero=True, rangeMin=20),
    color="values:N",
)

c1 = base.mark_arc(innerRadius=20, stroke="#fff")

c2 = base.mark_text(radiusOffset=10).encode(text="values:Q")

c1 + c2

alt.LayerChart(...)

In [80]:
# https://altair-viz.github.io/gallery/stacked_bar_chart.html

import altair as alt
from vega_datasets import data

source = data.barley()

alt.Chart(source).mark_bar().encode(
    x='variety',
    y='sum(yield)',
    color='site'
)

alt.Chart(...)

# EMISIONES DE CO2

In [1]:
# https://altair-viz.github.io/gallery/annual_weather_heatmap.html

import altair as alt
from vega_datasets import data

source = data.seattle_weather()

alt.Chart(source, title="Daily Max Temperatures (C) in Seattle, WA").mark_rect().encode(
    alt.X("date(date):O").title("Day").axis(format="%e", labelAngle=0),
    alt.Y("month(date):O").title("Month"),
    alt.Color("max(temp_max)").title(None),
    tooltip=[
        alt.Tooltip("monthdate(date)", title="Date"),
        alt.Tooltip("max(temp_max)", title="Max Temp"),
    ],
).configure_view(
    step=13,
    strokeWidth=0
).configure_axis(
    domain=False
)

alt.Chart(...)

In [91]:
import altair as alt
from vega_datasets import data

source = data.cars()

alt.Chart(source).transform_calculate(
    url='https://www.google.com/search?q=' + alt.datum.Name
).mark_point().encode(
    x='Horsepower:Q',
    y='Miles_per_Gallon:Q',
    color='Origin:N',
    href='url:N',
    tooltip=['Name:N', 'url:N']
)

alt.Chart(...)

In [92]:
# Ridgeline plot - https://altair-viz.github.io/gallery/ridgeline_plot.html

import altair as alt
from vega_datasets import data

source = data.seattle_weather.url

step = 20
overlap = 1

alt.Chart(source, height=step).transform_timeunit(
    Month='month(date)'
).transform_joinaggregate(
    mean_temp='mean(temp_max)', groupby=['Month']
).transform_bin(
    ['bin_max', 'bin_min'], 'temp_max'
).transform_aggregate(
    value='count()', groupby=['Month', 'mean_temp', 'bin_min', 'bin_max']
).transform_impute(
    impute='value', groupby=['Month', 'mean_temp'], key='bin_min', value=0
).mark_area(
    interpolate='monotone',
    fillOpacity=0.8,
    stroke='lightgray',
    strokeWidth=0.5
).encode(
    alt.X('bin_min:Q')
        .bin('binned')
        .title('Maximum Daily Temperature (C)'),
    alt.Y('value:Q')
        .axis(None)
        .scale(range=[step, -step * overlap]),
    alt.Fill('mean_temp:Q')
        .legend(None)
        .scale(domain=[30, 5], scheme='redyellowblue')
).facet(
    row=alt.Row('Month:T')
        .title(None)
        .header(labelAngle=0, labelAlign='left', format='%B')
).properties(
    title='Seattle Weather',
    bounds='flush'
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
).configure_title(
    anchor='end'
)

alt.FacetChart(...)